## 1.数据预处理

Data Preprocessing

In [31]:
# 控制调试输出的开关
DEBUG = not False

### 1.1 配置参数 
> 需根据实际文件路径修改
- 需要解析 PDF 文件路径
- 输出文件 CSV 路径
- 输出文件 CSV 表头

In [ ]:
CONST_PDF_PATH = "./assets/2024年四川省重点项目名单.pdf"  
CONST_OUTPUT_CSV = "./outputs/2024年四川重点项目解析结果.csv"
CONST_ORIGIN_HEADERS = ["总序号", "分序号", "项目名称", "建设地址"]
CONST_TARGET_HEADERS = ["项目名称", "建设地址", "一级项目领域", "二级项目领域", "项目类型"]


### 1.2 使用 pdfplumber 解析 PDF 页面数据

In [53]:
import pdfplumber
import re

# 申明一个(上下文)变量，一级项目领域，类型为字符串
Level_1_Project_Domain = ""
Level_2_Project_Domain = ""
Level_3_Project_Domain = ""

def extract_table_from_pdf(pdf_path, target_headers):
    all_tables_data = []  # 存储所有页面的表格数据

    with pdfplumber.open(pdf_path) as pdf:
        # 获取实际总页数
        total_pages = len(pdf.pages)  
        
        # 遍历PDF的每一页
        for page_num in range(0, total_pages):
        # for page_num in range(0, 10):
            page = pdf.pages[page_num]
            print(f"正在解析第{page_num + 1}页...")
            
            # 提取当前页所有表格
            table = page.extract_table(
                # 参考：
                # https://github.com/jsvine/pdfplumber?tab=readme-ov-file#table-extraction-settings
                table_settings={
                    "vertical_strategy": "lines",  # 按竖线识别列边界
                    "horizontal_strategy": "lines",  # 按横线识别行边界
                    "text_y_tolerance": 10,  # 文本垂直方向容错
                    "text_x_tolerance": 5,   # 文本水平方向容错
                }
            )
            
            # 使用新函数处理表格数据
            table_data = process_table_rows(table, target_headers)

            # 将处理后的数据添加到总数据列表中
            all_tables_data.extend(table_data)
    
    return all_tables_data

def process_table_rows(table, target_headers):
    """
    处理单个表格的所有行数据
    
    Args:
        table: 单个表格的数据（二维列表）
        target_headers: 目标表头列表
        
    Returns:
        tuple: (处理后的表头计数, 该表格处理后的数据列表)
    """
    table_data = []  # 存储当前表格处理后的数据

    for index, row in enumerate(table):

        # 格式为 target_headers 
        refined_data = []

        if not DEBUG:
            print(f"正在处理第 {index + 1} 行：{row}")

        # 跳过表头行
        if index < 2:
            continue  

        # 将行解构为单元格，以变量形式：总序号、分序号、项目名称、建设地址
        total_seq_cell, sub_seq_cell, project_name_cell, construction_address_cell = [cell.strip() if cell else "" for cell in row]
        project_name_cell = remove_after_first_paren(project_name_cell.replace("\n", ""))

        # total_seq_cell 为 '一二三四五' 其中一个汉字，则使用 project_name_cell 作为 一级项目领域
        global Level_1_Project_Domain 
        global Level_2_Project_Domain 
        global Level_3_Project_Domain 

        if re.match(r'^[一二三四五]$', total_seq_cell):
            Level_1_Project_Domain = project_name_cell  # 更新 1 级项目领域
            Level_2_Project_Domain = ""
            Level_3_Project_Domain = ""
            continue

        if re.match(r'^（[一二三四五]）$', total_seq_cell):
            Level_2_Project_Domain = project_name_cell  # 更新 2 级项目领域
            Level_3_Project_Domain = ""
            continue

        if total_seq_cell in ['I', 'II', 'III', 'IV', 'V', 'VI']:
            Level_3_Project_Domain = project_name_cell  # 更新 3 级项目领域
            continue

        address_city = construction_address_cell.replace("\n", " ")

        cleaned_row = [project_name_cell, address_city, Level_1_Project_Domain, Level_2_Project_Domain]

        if DEBUG:
            print(f"🌈已经处理第 {index + 1} 行：{cleaned_row}")

        # 清理每个单元格的多余空格和换行符
        cleaned_row = [cell.strip().replace("\n", "") if cell else "" for cell in row]
        
        if not DEBUG:
            print(f"清理后的行数据：{cleaned_row}")

        # 去重重复表头
        is_header = all(col in cleaned_row for col in target_headers)
        if is_header:
            table_data.append(cleaned_row)  # 保留第一次出现的表头
        else:
            # 筛选有效数据
            total_seq = cleaned_row[0] if len(cleaned_row) > 0 else ""
            project_name = cleaned_row[2] if len(cleaned_row) > 2 else ""
            
            # 排除分类标题
            if (total_seq.isdigit() or total_seq == "") and project_name != "" and "项目" in project_name:
                table_data.append(cleaned_row)
                
    return table_data

def remove_after_first_paren(text):
    # 使用正则表达式匹配第一个左括号及其后面的所有内容并替换为空字符串
    return re.sub(r'（.*', '', text)
    
# 提取表格数据
all_tables_data = extract_table_from_pdf(CONST_PDF_PATH, CONST_ORIGIN_HEADERS) 
    

正在解析第1页...
🌈已经处理第 3 行：['合计：504项', '', '', '']
🌈已经处理第 6 行：['国家实验室', '四川 天府新区', '基础设施项目', '新型基础设施']
🌈已经处理第 7 行：['极深地下极低辐射本底前沿物理实验设施', '凉山州 冕宁县', '基础设施项目', '新型基础设施']
🌈已经处理第 8 行：['光场调控装置及地方配套项目', '四川 天府新区', '基础设施项目', '新型基础设施']
🌈已经处理第 9 行：['多态耦合轨道交通动模试验平台', '四川 天府新区', '基础设施项目', '新型基础设施']
🌈已经处理第 10 行：['中国地震科学实验场', '有关市（州）', '基础设施项目', '新型基础设施']
🌈已经处理第 11 行：['柔性基底微纳结构成像系统研究装置', '四川 天府新区', '基础设施项目', '新型基础设施']
🌈已经处理第 12 行：['红外太赫兹自由电子激光装置', '成都市 双流区', '基础设施项目', '新型基础设施']
🌈已经处理第 13 行：['民航科技创新示范区一期工程', '成都市 东部新区', '基础设施项目', '新型基础设施']
正在解析第2页...
🌈已经处理第 3 行：['四川天府新区585研发中心项目', '四川 天府新区', '基础设施项目', '新型基础设施']
🌈已经处理第 4 行：['达州数字经济产业园', '达州市 高新区', '基础设施项目', '新型基础设施']
🌈已经处理第 5 行：['简阳市立昂云数据一号基地项目', '成都市 简阳市', '基础设施项目', '新型基础设施']
🌈已经处理第 6 行：['德阳“云上天府”大数据产业园', '德阳市 旌阳区', '基础设施项目', '新型基础设施']
🌈已经处理第 7 行：['成都东部新区简州新城阿里云西部云计算中心及数据服务基地项目', '成都市 东部新区', '基础设施项目', '新型基础设施']
🌈已经处理第 8 行：['成都高新区中国移动科研枢纽中心工程', '成都市 高新区', '基础设施项目', '新型基础设施']
🌈已经处理第 9 行：['雅安大数据中心项目', '雅安市 经开区', '基础设施项目', '新型基础设施']
🌈已经处理第 10 行：['四川广电数字城市“双千

### 1.3 使用 pandas 数据清洗

转换为 DataFrame 并清洗

In [41]:
import pandas as pd

def convert_to_dataframe(all_tables_data):
    # 将列表转为 DataFrame，用目标表头命名列
    df = pd.DataFrame(all_tables_data[1:], columns=all_tables_data[0])  # 0行是表头，1行后是数据

    # 进一步清洗：删除全空行、处理缺失值
    df = df.dropna(how="all")  # 删除全空行

    # 确保列存在再进行填充操作
    if "总序号" in df.columns:
        df["总序号"] = df["总序号"].fillna("")  # 总序号为空的填充为空字符串（如子项目）
    if "分序号" in df.columns:
        df["分序号"] = df["分序号"].fillna("")  # 分序号为空的填充为空字符串

    # 添加“项目类型”列
    if "总序号" in df.columns:
        df["项目类型"] = df["总序号"].apply(lambda x: "续建" if (str(x).isdigit() and int(x) <= 504) else "新开工")
    else:
        print("警告：未找到'总序号'列，跳过'项目类型'生成。")
        df["项目类型"] = ""
    
    return df

### 1.4 保存结果（CSV）

In [9]:
def save_results(df, output_csv):
    # 保存为 CSV（兼容更多工具，如 Excel、Python读取）
    # utf-8-sig 解决中文乱码
    df.to_csv(output_csv, index=False, encoding="utf-8-sig") 
    return output_csv


### 数据处理主函数运行 (main)
> 组织各个步骤函数运行

In [ ]:
from datetime import datetime

def mainDataPreprocess():
    # 配置参数
    pdf_path, output_csv, target_headers = prepare_params()
    
    # 提取表格数据
    all_tables_data = extract_table_from_pdf(pdf_path, target_headers)
    
    # 转换为DataFrame
    df = convert_to_dataframe(all_tables_data)
    
    # 保存结果
    saved_path = save_results(df, output_csv)

    # 获取当前时间
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 输出结果信息
    print(f"解析完成({current_time})！共提取{len(df)}条项目数据，已保存至：")
    print(f"- CSV: {saved_path}")
    
    # 打印前5条数据验证结果
    print("\n解析结果预览（前5条）：")
    print(df.head())

# 运行主程序
if __name__ == "__main__":
    mainDataPreprocess()

正在解析第1页...
正在处理行：['序号', None, '项目名称', '建设地址']
正在处理行：['总序号', '分序号', None, None]
正在处理行：['', '', '合计：504项', '']
正在处理行：['一', '', '基础设施项目（215项）', '']
正在处理行：['（一）', '', '新型基础设施（21项）', '']
正在处理行：['1', '1', '国家实验室', '四川\n天府新区']
正在处理行：['2', '2', '极深地下极低辐射本底前沿物理实验设施', '凉山州\n冕宁县']
正在处理行：['3', '3', '光场调控装置及地方配套项目', '四川\n天府新区']
正在处理行：['4', '4', '多态耦合轨道交通动模试验平台', '四川\n天府新区']
正在处理行：['5', '5', '中国地震科学实验场（在川部分）', '有关市（州）']
正在处理行：['6', '6', '柔性基底微纳结构成像系统研究装置', '四川\n天府新区']
正在处理行：['7', '7', '红外太赫兹自由电子激光装置', '成都市\n双流区']
正在处理行：['8', '8', '民航科技创新示范区一期工程', '成都市\n东部新区']
警告：未找到'总序号'列，跳过'项目类型'生成。
解析完成(2025-11-03 23:30:31)！共提取0条项目数据，已保存至：
- CSV: ./outputs/2024年四川重点项目解析结果.csv

解析结果预览（前5条）：
Empty DataFrame
Columns: [3, 3, 光场调控装置及地方配套项目, 四川天府新区, 项目类型]
Index: []
